In [ ]:
import nbimporter
import random
from imports import *
from functions_uc3 import *
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data_ume_ = pd.read_csv("./YOUR_PATH/YOUR_FILE.csv", delimiter=';')

In [ ]:
# data preprocessing
data_ume_scaled, scaler_ume = process_use_case_3_data_with_scaling(data_ume_, start_date_ume, end_date_ume)

In [ ]:
# Recreate rolling windows 
X, y, prediction_start_dates,X_df,y_df,full_data = create_rolling_windows_with_weekday_prediction(
    data_ume_scaled, target_col="shift_count", lookback=42, horizon=28, prediction_days=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday']
)
valid_series_dict_full, test_series_dict_full, train_series_dict_full = ts_dictionary(full_data, X_df, y_df)


In [ ]:
def time_series_kfold_cv_simplified(model_class, valid_series_dict_full,valid_series_dict_full_covars, scaler, 
                                    prediction_length, input_chunk_length, output_chunk_length, 
                                    epochs=50, n_splits=5, random_state=33):
    """
    Simplified k-fold cross-validation using `valid_series_dict_full` for both training and validation.
    
    Parameters:
    - model_class: A callable that returns a new model instance (e.g., `lambda **kwargs: NBEATSModel(...)`).
    - valid_series_dict_full: Dictionary of all available time series.
    - scaler: Scaler used for rescaling the data.
    - prediction_length: Prediction horizon.
    - input_chunk_length: Input chunk length for the model.
    - output_chunk_length: Output chunk length for the model.
    - epochs: Number of epochs to train the model.
    - n_splits: Number of folds.
    - random_state: Random state for reproducibility.
    
    Returns:
    - Fold-wise metrics and aggregated metrics.
    """
    keys = list(valid_series_dict_full.keys())
    kf = KFold(n_splits=n_splits, shuffle=True)

    fold_results = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(keys)):
        print(f"Fold {fold + 1}/{n_splits}")
        print(f"Hello 1")
        # Use keys to create train and validation sets
        train_keys = [keys[i] for i in train_idx]
        valid_keys = [keys[i] for i in valid_idx]

        train_fold_series = {k: train_series_dict_full[k] for k in train_keys}
        train_fold_series_covars = {k: train_series_dict_full_covars[k] for k in train_keys}
        valid_fold_series = {k: valid_series_dict_full[k] for k in valid_keys}
        valid_fold_series_covars = {k: valid_series_dict_full_covars[k] for k in valid_keys}
        print('ALL GOOD SO FAR')
        # Ensure no overlap between train and validation
        train_fold_series, valid_fold_series,valid_fold_series_covars = prepare_train_validation_split(
            train_fold_series, valid_fold_series,valid_fold_series_covars, input_chunk_length, output_chunk_length
        )
        print(f"Hello 2")
        print(f"Fold {fold + 1}: {len(train_fold_series)} training series, {len(valid_fold_series)} validation series.")

        # Skip if no training series are left
        if len(train_fold_series) == 0:
            print(f"Skipping Fold {fold + 1} due to empty training set.")
            continue

        # Initialize and train the model
        model = model_class(
            input_chunk_length=input_chunk_length,
            output_chunk_length=prediction_length,
            random_state=random_state,
            epochs=epochs,
        )
        try:
            model.fit(series=list(train_fold_series.values()),past_covariates=list(train_fold_series_covars.values()),val_series=list(valid_fold_series.values()),val_past_covariates=list(valid_fold_series_covars.values()))
        except Exception as e:
            print(f"Error fitting model in Fold {fold + 1}: {e}")
            continue

        # Truncate validation series to exclude the last `prediction_length` entries
        truncated_valid_fold_series = {
            key: series[:-prediction_length] for key, series in valid_fold_series.items()
        }
        truncated_valid_fold_series_covars = {
            key: series[:-prediction_length] for key, series in valid_fold_series_covars.items()
        }
#
        # Generate predictions for truncated validation series
        try:
            pred = model.predict(n=prediction_length, series=list(truncated_valid_fold_series.values()),past_covariates=list(truncated_valid_fold_series_covars.values()),num_samples=500)
            pred_arrays = np.array([ts.all_values() for ts in pred]).squeeze(axis=2)
            pred_medians = np.median(pred_arrays, axis=2)
        except Exception as e:
            print(f"Error predicting in Fold {fold + 1}: {e}")
            continue

        # Align actual values with predictions
        actual_values = []
        predicted_values = []

        for key, pred_series in zip(valid_keys, pred_medians):
            actual_values.append(valid_series_dict_full[key].values()[-prediction_length:].flatten())
            predicted_values.append(pred_series)

        # Convert to NumPy arrays for consistency
        actual_values_np = np.array(actual_values)
        predicted_values_np = np.array(predicted_values)

        # Rescale predictions
        actual_values_rescaled = scaler.inverse_transform(actual_values_np)
        predicted_values_rescaled = scaler.inverse_transform(predicted_values_np)

        # Evaluate metrics
        metrics = bootstrap_metrics_train(actual_values_rescaled.flatten(), predicted_values_rescaled.flatten())

        # Store metrics for this fold
        fold_results.append({
            "fold": fold + 1,
            "metrics": metrics,
            "predictions": predicted_values_rescaled,
            "actuals": actual_values_rescaled,
        })
        print(f"Fold {fold + 1} results: {metrics}")

    # Aggregate results
    if fold_results:
        aggregated_metrics = {metric: np.mean([fold["metrics"][metric][0] for fold in fold_results]) 
                              for metric in ["mae", "mse", "rmse", "mape", "smape"]}
        print("Aggregated cross-validation metrics MEAN:")
        print(aggregated_metrics)
        aggregated_metrics_05 = {metric: np.mean([fold["metrics"][metric][1] for fold in fold_results]) 
                              for metric in ["mae", "mse", "rmse", "mape", "smape"]}
        print("Aggregated cross-validation metrics CI-LOW:")
        print(aggregated_metrics_05)
        aggregated_metrics_95 = {metric: np.mean([fold["metrics"][metric][2] for fold in fold_results]) 
                              for metric in ["mae", "mse", "rmse", "mape", "smape"]}
        print("Aggregated cross-validation metrics CI-UP:")
        print(aggregated_metrics_95)
    else:
        print("No valid folds to aggregate metrics.")
        aggregated_metrics = {}

    return fold_results, aggregated_metrics


# Global variables to track the best model and score
best_model = None
best_score = float("inf")

def objective(trial):
    global best_model, best_score
    print(f"Trial {trial.number}: Starting")

    mae_all_encounters = {}

    # Define search space for hyperparameters
    lookback = trial.suggest_int("lookback", 7, 22)
    layer_widths_value = trial.suggest_int("layer_widths", 32, 512)
    num_blocks = trial.suggest_int("num_blocks", 1, 4)
    dropout = trial.suggest_uniform("dropout", 0.0, 0.5)
    batch_size = trial.suggest_int("batch_size", 32, 256)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # Layer widths list
    layer_widths = [layer_widths_value] * num_blocks

    # Define early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=5, verbose=False)
    callbacks = [early_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    #print('multi_models', multi_models)
    #print('lags', lags)

    model = NBEATSModel(
        input_chunk_length=9,
        output_chunk_length=prediction_length,
        num_blocks=num_blocks,
        layer_widths=layer_widths,
        dropout=dropout,
        n_epochs=200,
        batch_size=batch_size,
        optimizer_kwargs={"lr": learning_rate},
        pl_trainer_kwargs=pl_trainer_kwargs,
        random_state=42,
        likelihood=QuantileRegression(),
        force_reset=True,
        save_checkpoints=False,
    )

    # Perform cross-validation
    fold_results, aggregated_metrics = time_series_kfold_cv_simplified(
        model_class=lambda **kwargs: model,
        valid_series_dict_full=valid_series_dict_full,
        scaler=scaler_ume,
        prediction_length=28,
        input_chunk_length=42,
        output_chunk_length=5,
        epochs=200,
        n_splits=5,
    )

    # Get mean MAE across folds
    mean_mae = aggregated_metrics.get("mae", float("inf"))

    # Update best model if current one is better
    if mean_mae < best_score:
        best_score = mean_mae
        best_model = model

    print(f"Trial {trial.number}: Finished with MAE = {mean_mae}")
    return mean_mae

def print_callback(study, trial):
    print(f"Best value so far: {study.best_value}, Best params: {study.best_trial.params}")

if __name__ == "__main__":
    # Set up Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, callbacks=[print_callback])

    # Save the best model
    if best_model is not None:
        best_model.save("YOUR_PATH")


